In [ ]:
import subprocess
import sys

COLAB = "google.colab" in sys.modules


def _install(package):
    if COLAB:
        answer = input(f"Install { package }? [y/n]:")
        if answer.lower() in ["y", "yes"]:
            subprocess.check_call(
                [sys.executable, "-m", "pip", "install", "--upgrade--quiet", package],
            )
            print(f"{ package } installed!")


def _colab_install_missing_deps(deps):
    import importlib

    for dep in deps:
        if importlib.util.find_spec(dep) is None:
            _install(dep)


# folium is outaded in colab and must be upgraded.
deps = ["ioos_metrics", "gliderpy", "folium"]
_colab_install_missing_deps(deps)

In [ ]:
from ioos_metrics.ioos_metrics import _ngdac_gliders as ngdac_gliders

query = {
    "min_time": "2023-06-01T00:00:00",
    "max_time": "2023-11-30T23:59:59",
    "min_lat": 10,
    "max_lat": 42,
    "min_lon": -99,
    "max_lon": -50,
}

In [ ]:
%%time

import pandas as pd

metadata_metrics = ngdac_gliders(**query)
metadata_metrics["time_coverage_start"] = pd.to_datetime(
    metadata_metrics["time_coverage_start"],
)
metadata_metrics["time_coverage_end"] = pd.to_datetime(
    metadata_metrics["time_coverage_end"],
)

We could save after the clean-ups below but let's keep it closer to gdutils for now and save as-is.

In [ ]:
_save_df = (
    metadata_metrics[
        [
            "glider",
            "wmo_id",
            "sea_name",
            "time_coverage_start",
            "time_coverage_end",
            "days",
            "num_profiles",
            "institution",
            "acknowledgment",
        ]
    ]
    .reset_index(
        names=["dataset_id"],
    )
    .sort_values(
        by="dataset_id",
        key=lambda x: x.str.lower(),
    )
    .rename(
        columns={
            "time_coverage_start": "start_date",
            "time_coverage_end": "end_date",
            "sea_name": "deployment_area",
        },
    )
)

_save_df["start_date"] = _save_df["start_date"].dt.strftime("%m-%d-%Y")
_save_df["end_date"] = _save_df["end_date"].dt.strftime("%m-%d-%Y")
_save_df["days"] = [d.days for d in _save_df["days"]]

_save_df.to_csv(
    path_or_buf="glider_metadata_metrics.csv",
    index=False,
)

In [ ]:
gdf = metadata_metrics.copy()

In [ ]:
import geopandas

colors = {
    1: ["#E6E6FA", "Jan"],
    2: ["#D2691E", "Feb"],
    3: ["#DC143C", "Mar"],
    4: ["#7FFFD4", "Apr"],
    5: ["#006400", "May"],
    6: ["#FF8C00", "Jun"],
    7: ["#2F4F4F", "Jul"],
    8: ["#FF1493", "Aug"],
    9: ["#FF69B4", "Sep"],
    10: ["#7FFF00", "Oct"],
    11: ["#BA55D3", "Nov"],
    12: ["#EE82EE", "Dec"],
}


# gdf["time_coverage_start"] = pd.to_datetime(gdf["time_coverage_start"])
# gdf["time_coverage_end"] = pd.to_datetime(gdf["time_coverage_end"])
gdf = gdf.reset_index(names="glider deployment")
gdf = gdf.rename({"track": "geometry"}, axis=1)
style = [
    {
        "lineColor": colors[date.month][0],
        "fillColor": colors[date.month][0],
        "weight": 4,
        "color": colors[date.month][0],
    }
    for date in gdf["time_coverage_start"]
]
gdf["style"] = style

gdf = geopandas.GeoDataFrame(gdf)
gdf = gdf.set_crs(epsg=4326)

The `metadata_metrics` is DataFrame with some basic metadata from all gliders in the query.
Normalize `sea_names` column.

In [ ]:
fix_names = {
    "mid atlantic bight": "mid-atlantic bight",
    "north atlantic ocean": "north atlantic",
    "northwest atlantic ocean": "northwest atlantic",
}

metadata_metrics["sea_name"] = metadata_metrics["sea_name"].str.lower().str.strip().replace(fix_names)
metadata_metrics["sea_name"].value_counts().plot.barh();

In [ ]:
short_names = {
    "bermuda institute of ocean sciences": "Bermuda IOS",
    "ensenada center for scientific research and higher education and gulf of mexico ocean observing system (gcoos)": "GCOOS",  # noqa: E501
    "mote marine laboratory": "MML",
    "national oceanic and atmospheric administration (noaa) / atlantic oceanographic and meteorological laboratory (aoml), integrated ocean observing system (ioos), and gulf coast ocean observing system (gcoos)": "GCOOS",  # noqa: E501
    "national oceanic and atmospheric administration (noaa) / atlantic oceanographic and meteorological laboratory (aoml), and integrated ocean observing system (ioos) / caribbean regional association for coastal ocean observing (caricoos)": "CARICOOS",  # noqa: E501
    "naval oceanographic office": "NOO",
    "rutgers university": "RU",
    "skidaway institute of oceanography": "SIO",
    "stony brook university": "SBO",
    "texas a&m university": "Texas A&M",
    "united states naval research laboratory": "NRL",
    "university of connecticut": "UC",
    "university of delaware": "UD",
    "university of louisiana at lafayette": "ULL",
    "university of south florida": "USF",
    "university of south florida college of marine science ocean technology group": "USF",
    "university of southern mississippi": "USM",
    "university of the virgin islands, rutgers university": "UVI",
    "virginia institute of marine science": "VIMS",
    "virginia institute of marine science - the college of william & mary": "VIMS",
    "woods hole oceanographic institution": "WHOI",
}

metadata_metrics["institution"] = metadata_metrics["institution"].str.lower().str.strip().replace(short_names)
metadata_metrics["institution"].value_counts().plot.barh();

In [ ]:
from branca.element import MacroElement, Template

color_legend = []
for color, month in colors.values():
    color_legend.append(
        f"""<li><span style='background:{color};opacity:0.7;'></span>{month}</li>""",
    )

color_legend = "\n".join(color_legend)


template = f"""
{{% macro html(this, kwargs) %}}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>jQuery UI Draggable - Default functionality</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>

  <script>
  $( function() {{
    $( "#maplegend" ).draggable({{
                    start: function (event, ui) {{
                        $(this).css({{
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        }});
                    }}
                }});
}});

  </script>
</head>
<body>


<div id='maplegend' class='maplegend'
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>

<div class='legend-title'>Legend</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    {color_legend}
  </ul>
</div>
</div>

</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {{
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }}
  .maplegend .legend-scale ul {{
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }}
  .maplegend .legend-scale ul li {{
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }}
  .maplegend ul.legend-labels li span {{
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }}
  .maplegend .legend-source {{
    font-size: 80%;
    color: #777;
    clear: both;
    }}
  .maplegend a {{
    color: #777;
    }}
</style>
{{% endmacro %}}"""

macro = MacroElement()
macro._template = Template(template)  # noqa: SLF001

In [ ]:
import folium

m = folium.Map()
for _, glider in gdf.iterrows():
    exec(f"style_function = lambda x: {glider['style']}")  # noqa: S102
    # See https://github.com/python-visualization/folium/issues/1932
    # style_function = lambda x: glider['style']
    folium.GeoJson(
        data=glider["geometry"],
        style_function=style_function,  # noqa: F821
        tooltip=glider["glider deployment"],
        popup=f"{glider['time_coverage_start']} -- {glider['time_coverage_end']}",
    ).add_to(m)
m.fit_bounds(m.get_bounds())

m.get_root().add_child(macro)
m

Using `metadata_metrics` we can compute the glider days, number of profiles, deployments, and more.

In [ ]:
n_days = metadata_metrics["days"].sum().days
n_profiles = metadata_metrics["num_profiles"].sum()
n_deployments = len(metadata_metrics)

In [ ]:
print(
    f"""\
For the {query=} we got:\n
{n_days=}
{n_profiles=}
{n_deployments=}
""",
)